In [4]:
from torch_geometric.data import Data as pyg_Data
from torch_geometric.data import Batch as pyg_Batch
from torch_geometric.loader import DataLoader as pyg_Loader
import torch_geometric.nn as pyg_nn

import torch
import torch.nn.functional as F
import torch.nn as nn


In [5]:
adj = []

for _ in range(8):

    mat = torch.rand((10, 10)) < 0.5
    adj.append(mat.to(torch.int))

adj = torch.stack(adj)
print(adj.shape)

print(adj[0])

torch.Size([8, 10, 10])
tensor([[1, 1, 1, 0, 1, 1, 0, 0, 0, 1],
        [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 0, 0, 1, 1, 0, 1, 1, 0, 1],
        [0, 0, 1, 1, 1, 0, 0, 1, 1, 0],
        [0, 0, 1, 0, 0, 0, 0, 1, 0, 1],
        [1, 1, 1, 0, 1, 0, 0, 0, 1, 0],
        [1, 1, 0, 0, 1, 0, 1, 1, 0, 0],
        [1, 1, 1, 0, 1, 1, 0, 1, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 1, 0, 0]], dtype=torch.int32)


In [6]:
features = torch.randn((8, 10, 32)) # batch, num_node, num_features
features.shape

torch.Size([8, 10, 32])

In [15]:
def convert_to_edgeindex(matrix, features):

    batch_size, num_nodes, _ = matrix.size()
    edge_indices = []
    num_nodes = [num_nodes for i in range(batch_size)]

    for i in range(batch_size):
        adj = matrix[i]
        adj_coo = adj.to_sparse().coalesce()
        edge_index = adj_coo.indices()
        edge_indices.append(edge_index)

    # print(edge_indices[0])
    # print(edge_indices[1].shape)
    
    graph_data = [pyg_Data(edge_index=e, x=x, test = 'test') for e, x in zip(edge_indices, features)]
    return graph_data

    # # for first graph
    # g1 = graph_data[0]
    # print(g1)

    # g_loader = pyg_Loader(graph_data, batch_size=8)
    # return g_loader

In [16]:
data_list = convert_to_edgeindex(adj, features)
batch = pyg_Batch.from_data_list(data_list)
batch

DataBatch(x=[80, 32], edge_index=[2, 396], test=[8], batch=[80], ptr=[9])

In [9]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels) -> None:
        super(GCN, self).__init__()
        self.conv1 = pyg_nn.GCNConv(in_channels, hidden_channels)
        self.conv2 = pyg_nn.GCNConv(hidden_channels, out_channels)


    def forward(self, x, edge_index):

        x = self.conv1(x, edge_index)
        x = self.conv2(x, edge_index)
        return x

In [13]:
model = GCN(32, 16, 1)
print(batch.x.shape)
print(batch.edge_index.shape)
output = model(batch.x, batch.edge_index)

torch.Size([80, 32])
torch.Size([2, 396])


In [14]:
output.shape

torch.Size([80, 1])